In [15]:
#from shutil import copyfile

# copy file into the working directory (make sure it has .py suffix)
#copyfile(src = "../input/proxy-training-s2s/Proxy_training_S2S.py", dst = "../working/Proxy_training_S2S.py")

# import all functions
#from Proxy_training_S2S import ProxyTraining, DataLoad
#!pip install openpyxl
#!pip install torch

In [16]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os
from transformers import get_scheduler
from tqdm.auto import tqdm
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration
from datasets import load_metric
from sklearn.model_selection import KFold
import copy
import time 

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

In [17]:
class DataLoad:

    """

    Class to read in the BABE Dataset for the ProxyTask

    Uses the final labels SG1 for mbic and final labels SG2 for babe and combines them

    """

    @staticmethod

    def read_babe():


        df = pd.read_excel("../input/babe-media-bias-annotations-by-experts/data/final_labels_SG2.xlsx")

        lst = []

        for index, row in df.iterrows():

            if row['label_bias'] == "No agreement":

                pass

            else:

                sub_dict = {'text': 'biased or non-biased:' + row['text']}

                if row['label_bias'] == "Biased":

                    sub_dict['label'] = "Biased"

                elif row['label_bias'] == "Non-biased":

                    sub_dict['label'] = "Non-biased"

                lst.append(sub_dict)

        return pd.DataFrame(lst)

In [18]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda:1' if cuda.is_available() else 'cpu'

In [19]:
class YourDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }


In [20]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        #print(data)
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        #print(y_ids)
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        #print(lm_labels.shape)
        #print(lm_labels)
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)
        #print(ids, mask, y_ids, lm_labels)
        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            labels=y
        )
        loss = outputs[0]

        #if _ % 10 == 0:
         #   training_logger.add_row(str(epoch), str(_), str(loss))
          #  console.print(training_logger)
        lr_scheduler.step()
        progress_bar.update(1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [21]:
def validate(epoch, tokenizer, model, device, loader):

    """
    Function to evaluate model for predictions

    """
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            #print(data)
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
            lm_labels = y[:,1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            # y_biased, y_nonbiased = [ 2106,     9,  3843,     1,     0], [ 5388,    18, 15500,  3843,     1] #T5
            y_biased, y_nonbiased = [25352,   102,     0,     0,     0], [ 2512,  1011, 25352,   102,     0] # ProphetNet
            prediction_lst, probs_biased, probs_nonbiased = [], [], []
            for i in range(len(ids)):
                output = model(input_ids=ids[i].unsqueeze(0),
                    attention_mask=mask[i].unsqueeze(0),
                    labels=y[i].unsqueeze(0))
                probs = F.softmax(output[1], dim=1)
                for j in range(len(probs[0])):
                    probs_biased.append(probs[0][j][y_biased[j]])
                    probs_nonbiased.append(probs[0][j][y_nonbiased[j]])
                probs_b = sum(probs_biased)/len(probs_biased)
                probs_nb = sum(probs_nonbiased)/len(probs_nonbiased)
                if probs_b > probs_nb:
                    prediction_lst.append(1)
                else:
                    prediction_lst.append(0)
            # target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            #if _%10==0:
             #   console.print(f'Completed {_}')

            predictions.extend(prediction_lst)
            actuals.extend(y)
        return predictions, actuals


In [22]:
def ModelTrainer(
    dataframe, source_text, target_text, model_params
):

    """
    Model trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True
    print(f"""[Model]: Loading {model_params["MODEL"]}...\n""")
    transformer_model = ProphetNetForConditionalGeneration.from_pretrained(model_params["MODEL"], use_cache = False)
    dataframe = dataframe[[source_text, target_text]]

    # tokenzier for encoding the text
    tokenizer = ProphetNetTokenizer.from_pretrained(model_params["MODEL"])
    splits = KFold(n_splits=5, shuffle=True, random_state=42)  # k-fold k=5 as in the BABE paper
    overall_f1, results = [], [] # list for all f1 scores of the folds
    for fold, (train_ids, val_ids) in enumerate(splits.split(np.arange(len(dataframe)))):

        # Defining the model. We are using a base model and added a Language model layer on top for generation of Summary.
        # Further this model is sent to device (GPU/TPU) for using the hardware.
        
           
        model = copy.deepcopy(transformer_model)
        model.to(device)
        train_dataset = pd.DataFrame(dataframe.iloc[train_ids]).reset_index(drop=True)
        val_dataset = pd.DataFrame(dataframe.iloc[val_ids]).reset_index(drop=True)
    

        print(f"FULL Dataset: {dataframe.shape}")
        print(f"TRAIN Dataset: {train_dataset.shape}")
        print(f"TEST Dataset: {val_dataset.shape}\n")

        # Creating the Training and Validation dataset for further creation of Dataloader
        training_set = YourDataSetClass(
            train_dataset,
            tokenizer,
            model_params["MAX_SOURCE_TEXT_LENGTH"],
            model_params["MAX_TARGET_TEXT_LENGTH"],
            source_text,
            target_text,
        )
        val_set = YourDataSetClass(
            val_dataset,
            tokenizer,
            model_params["MAX_SOURCE_TEXT_LENGTH"],
            model_params["MAX_TARGET_TEXT_LENGTH"],
            source_text,
            target_text,
        )

        # Defining the parameters for creation of dataloaders
        train_params = {
            "batch_size": model_params["TRAIN_BATCH_SIZE"],
            "shuffle": True,
            "num_workers": 0,
        }

        val_params = {
            "batch_size": model_params["VALID_BATCH_SIZE"],
            "shuffle": False,
            "num_workers": 0,
        }

        # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
        training_loader = DataLoader(training_set, **train_params)
        val_loader = DataLoader(val_set, **val_params)

        # Defining the optimizer that will be used to tune the weights of the network in the training session.
        optimizer = torch.optim.Adam(
            params=model.parameters(), lr=model_params["LEARNING_RATE"]
        )

        # Training loop
        print(f"[Initiating Fine Tuning]...\n")
        num_training_steps = model_params["TRAIN_EPOCHS"] * len(training_loader)  # for the progress bar
        global lr_scheduler 
        lr_scheduler = get_scheduler(
            "linear",
            optimizer=optimizer,
            num_warmup_steps=0,
            num_training_steps=num_training_steps
        )
        global progress_bar 
        progress_bar = tqdm(range(num_training_steps))
        
        test_f1_lst = []

        for epoch in range(model_params["TRAIN_EPOCHS"]):
            #TRAINING
            print(f"Epoch {epoch}")
            train(epoch, tokenizer, model, device, training_loader, optimizer)

            #EVALUATION
            predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
            #final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
            #print(predictions, actuals)
            target_transformed = []
            #print(actuals)
            for i in actuals:
                if i.tolist() ==  [ 2512,  1011, 25352,   102,     0]:
                    target_transformed.append(0)
                elif i.tolist() == [25352,   102,     0,     0,     0]:
                    target_transformed.append(1)
                else:
                    target_transformed.append(2)  # only a workaround for now
                    
            #print(predictions, target_transformed)
            metric_f1 = load_metric("f1")
            metric_acc = load_metric("accuracy")
            metric_f1.add_batch(predictions=predictions, references=target_transformed)
            metric_acc.add_batch(predictions=predictions, references=target_transformed)

            test_f1, test_acc = metric_f1.compute(average='weighted'), metric_acc.compute()
            print(f"Fold: {fold} Epoch {epoch}: Avg test acc: {test_acc}, Avg test f1: {test_f1}")
            results.append(f"Fold: {fold} Epoch {epoch}: Avg test acc: {test_acc}, Avg test f1: {test_f1}")
            test_f1_lst.append(test_f1['f1'])
            
        overall_f1.append(max(test_f1_lst))
        
    avg_f1 = sum(overall_f1) / len(overall_f1)
    results.append(avg_f1)
    print(results)
    with open(str(time.time())+model_params["MODEL"]+'results.txt', 'w') as f:
            f.write(str(results))
    print(f'Overall avg f1: {avg_f1}')
    return avg_f1

In [23]:
model_params_t5 = {
    "MODEL": "t5-base",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 4,  # training batch size
    "VALID_BATCH_SIZE": 4,  # validation batch size
    "TRAIN_EPOCHS": 4,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 5e-5,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 5,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

In [24]:
model_params_prophetnet = {
    "MODEL": "microsoft/prophetnet-large-uncased",  # model_type
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TRAIN_EPOCHS": 4,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 5e-5,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 5,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

In [25]:
df = DataLoad.read_babe()
ModelTrainer(
    dataframe=df,
    source_text="text",
    target_text="label",
    model_params=model_params_prophetnet,
)

[Model]: Loading microsoft/prophetnet-large-uncased...



Downloading: 100%|██████████| 1.40k/1.40k [00:00<00:00, 1.92MB/s]
/beegfs/zhukova/.conda/envs/ml/lib/python3.9/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Downloading: 100%|██████████| 1.57G/1.57G [00:20<00:00, 74.6MB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 534kB/s] 
Downloading: 100%|██████████| 90.0/90.0 [00:00<00:00, 149kB/s]
Downloading: 100%|██████████| 141/141 [00:00<00:00, 230kB/s]
/beegfs/zhukova/.conda/envs/ml/lib/python3.9/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` inste

FULL Dataset: (3673, 2)
TRAIN Dataset: (2938, 2)
TEST Dataset: (735, 2)

[Initiating Fine Tuning]...



100%|██████████| 2940/2940 [31:06<00:00,  1.58it/s]
/beegfs/zhukova/.conda/envs/ml/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  0%|          | 1/1472 [00:00<03:50,  6.38it/s]

Epoch 0


...

 75%|███████▌  | 1105/1472 [11:25<1:03:49, 10.43s/it]

Fold: 4 Epoch 2: Avg test acc: {'accuracy': 0.6471389645776566}, Avg test f1: {'f1': 0.597343047683996}
Epoch 3


100%|██████████| 1472/1472 [14:42<00:00,  1.93it/s]  

Fold: 4 Epoch 3: Avg test acc: {'accuracy': 0.6471389645776566}, Avg test f1: {'f1': 0.597343047683996}
["Fold: 0 Epoch 0: Avg test acc: {'accuracy': 0.636734693877551}, Avg test f1: {'f1': 0.577809672385237}", "Fold: 0 Epoch 1: Avg test acc: {'accuracy': 0.636734693877551}, Avg test f1: {'f1': 0.577809672385237}", "Fold: 0 Epoch 2: Avg test acc: {'accuracy': 0.636734693877551}, Avg test f1: {'f1': 0.577809672385237}", "Fold: 0 Epoch 3: Avg test acc: {'accuracy': 0.636734693877551}, Avg test f1: {'f1': 0.577809672385237}", "Fold: 1 Epoch 0: Avg test acc: {'accuracy': 0.6136054421768707}, Avg test f1: {'f1': 0.5492760451944125}", "Fold: 1 Epoch 1: Avg test acc: {'accuracy': 0.6136054421768707}, Avg test f1: {'f1': 0.5492760451944125}", "Fold: 1 Epoch 2: Avg test acc: {'accuracy': 0.6136054421768707}, Avg test f1: {'f1': 0.5492760451944125}", "Fold: 1 Epoch 3: Avg test acc: {'accuracy': 0.6258503401360545}, Avg test f1: {'f1': 0.5681211346968096}", "Fold: 2 Epoch 0: Avg test acc: {'accur

FileNotFoundError: [Errno 2] No such file or directory: '1665198240.282313microsoft/prophetnet-large-uncasedresults.txt'